# 3_modeling_evaluation_suto

In [1]:
import os

# Biblioteca built-in para obtenção de combinações
from itertools import combinations

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import PolynomialFeatures

# Bibliotecas de modelos
from pycaret.regression import * 
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import (cross_val_score, RepeatedStratifiedKFold,
                                     RandomizedSearchCV, GridSearchCV, train_test_split)

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

path_data = "../data/raw/"
path_interim_data = "../data/interim/"

# Parâmetros estéticos dos gráficos:
set_palette = "Paired"
axes_style = "ticks"

linha = 50*"-"

In [12]:
df_data = pd.read_parquet(path_interim_data+"df_data_step_1_output.pqt")

X_train_encoded = pd.read_parquet(path_interim_data+"X_train_encoded_step_2.pqt")
X_test_encoded = pd.read_parquet(path_interim_data+"X_test_encoded_2_output.pqt")

X_train_encoded = pd.concat([X_train_encoded.drop(['car name'], axis=1), df_data.loc[X_train_encoded.index, "mpg"]], axis=1)
X_test_encoded = pd.concat([X_test_encoded.drop(['car name'], axis=1), df_data.loc[X_test_encoded.index, "mpg"]], axis=1)

print(f"""Dimensões do dataset de treino: {X_train_encoded.shape}
Dimensões do dataset de validação: {X_test_encoded.shape}""")

Dimensões do dataset de treino: (357, 9)
Dimensões do dataset de validação: (40, 9)


## Testando um vários modelos de regressão usando o PyCaret

In [ ]:
regression = setup(data=X_train_encoded, target='mpg')

best = compare_models(sort='MSE')

,Description,Value
0,session_id,7461
1,Target,mpg
2,Original Data,"(357, 9)"
3,Missing Values,False
4,Numeric Features,6
5,Categorical Features,2
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(249, 24)"


IntProgress(value=0, description='Processing: ', max=99)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,23:28:59
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,CatBoost Regressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,2.2445,9.6611,3.0742,0.8338,0.1158,0.0942,0.0550
ridge,Ridge Regression,2.3598,9.8254,3.1070,0.8298,0.1264,0.1040,0.0060
br,Bayesian Ridge,2.3620,9.8586,3.1122,0.8294,0.1263,0.1039,0.0060
lr,Linear Regression,2.3886,10.0850,3.1480,0.8241,0.1297,0.1059,0.4460
gbr,Gradient Boosting Regressor,2.3659,10.6558,3.2302,0.8196,0.1227,0.1004,0.0150
xgboost,Extreme Gradient Boosting,2.3748,11.3037,3.3299,0.8061,0.1260,0.1005,0.3090
rf,Random Forest Regressor,2.3928,11.3082,3.3173,0.8108,0.1245,0.1006,0.0580
ada,AdaBoost Regressor,2.8351,14.2557,3.7363,0.7615,0.1423,0.1211,0.0230
lightgbm,Light Gradient Boosting Machine,2.8588,16.0652,3.8865,0.7346,0.1423,0.1181,0.0100
omp,Orthogonal Matching Pursuit,3.1257,16.3911,4.0185,0.7187,0.1605,0.1369,0.0050
